In [1]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
load_dotenv()

c:\Users\shohr\miniconda3\envs\ctphacks24\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [33]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="hunter-clubs",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '9478ab63e2678dd87a29f93844e27b8e', 'Date': 'Thu, 22 Aug 2024 17:56:23 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [28]:
import json
data = json.load(open("club_json_files/hunter_clubs.json"))
data["clubs"]

[{'club_name': 'Accounting Society',
  'club_description': ['The Accounting Society of Hunter College aims to enhance the professional, academic, and social lives of its members. The fundamental goals of the society are to establish a professional culture and facilitate networking among the Hunter College accounting students, faculty, and industry professionals through speaker engagement events, career-building workshops, and community service. The Accounting Society strives to educate its members about current accounting and business issues, while simultaneously helping its members learn the hard and soft skills required to become successful business professionals and community leaders in today’s competitive business environment.'],
  'club_type': 'Academic',
  'club_links': {'website': 'https://www.accountingsocietyhc.com/',
   'instagram': 'https://www.instagram.com/accountingsocietyhc/',
   'linkedin': 'https://www.linkedin.com/in/accountingsocietyhc/',
   'email': 'accountingsocie

In [26]:
index = pc.Index("hunter-clubs")
index.delete(delete_all=True, namespace="ns1")

{}

In [30]:
openai = OpenAI()
processed_data = []

print(len(data["clubs"]))

for club in data["clubs"]:

    embedding_response = openai.embeddings.create(
        model="text-embedding-3-small",
        input=club["club_description"]
    )
    embedding = embedding_response.data[0].embedding

    # JSON Preprocessing
    media_list = []
    if "email" in club["club_links"]:
        media_list.append(club["club_links"]["email"])
    if "instagram" in club["club_links"]:
        media_list.append(club["club_links"]["instagram"])
    if "facebook" in club["club_links"]:
        media_list.append(club["club_links"]["facebook"])
    if "website" in club["club_links"]:
        media_list.append(club["club_links"]["website"])
    if "linktree" in club["club_links"]:
        media_list.append(club["club_links"]["linktree"])

    # normalized_name = to_ascii(club["club_name"])

    processed_data.append({
        "values": embedding,
        "id": club["club_name"],
        "metadata": {
            "club_type": club["club_type"],
            "description": club["club_description"],
            "media": media_list
        }
    })

101


In [32]:
processed_data[-1]

{'values': [-0.014798715,
  -0.030772693,
  -0.0016834438,
  0.03050886,
  -0.0053666327,
  0.020914879,
  -0.0126640545,
  -0.014522889,
  0.010139639,
  0.011872551,
  0.030916603,
  0.01574612,
  -0.038136072,
  0.006074189,
  0.00447919,
  -0.025448034,
  -0.037992164,
  -0.014235069,
  0.007855072,
  0.01903206,
  0.05996238,
  0.028254272,
  -0.012771986,
  0.01821657,
  0.0037596412,
  0.017425068,
  0.0015695153,
  0.0054176007,
  0.011356874,
  -0.013107776,
  0.017125256,
  -0.0011947504,
  -0.036169305,
  -0.025759839,
  0.05502148,
  0.0097618755,
  -0.01879221,
  -0.027126981,
  0.016657548,
  0.040294718,
  0.051471706,
  0.018132623,
  -0.004356267,
  -0.019439803,
  -0.003351897,
  -0.000693315,
  -0.009599977,
  -0.0080349585,
  0.0034328462,
  0.012807964,
  -0.014594843,
  0.00042123572,
  0.07550463,
  0.001939783,
  0.010217589,
  0.04746622,
  0.056316666,
  0.006026219,
  -0.03734457,
  0.024236795,
  -0.007891049,
  0.0008387238,
  -0.021370592,
  0.041206148,
 

In [34]:
index = pc.Index("hunter-clubs")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 101}

In [36]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 99}},
 'total_vector_count': 99}